# Generation of SQL code to fill BD

## TODO

- [ ] Check uniquiness of the key(s)
- [ ] Connect partial keys with existing tables
- [ ] Make data length flexible

Ds to be used from parsing:

- tableName - str
- list of columns: {str(name): dtype(str)}


In [1]:
!ls

fill.sql			   README.md   tablesUpdate.sql
GeneratorOfSQLDataGenerator.ipynb  tables.sql


In [2]:
import re
import names
import string 
import random 
import exrex
import random
from datetime import datetime, timedelta
import numpy as np

In [3]:
file = open("tablesUpdate.sql", "r")
full_text = file.read()
print(full_text)

CREATE TABLE EmployeeAccount (
	password VARCHAR(16) NOT NULL,
	login VARCHAR(16) NOT NULL,
	education VARCHAR(1024) NOT NULL,
	date_of_start_of_career DATE NOT NULL,
	surname VARCHAR(64) NOT NULL,
	name VARCHAR(64) NOT NULL,
	phone VARCHAR(11) NOT NULL,
	email VARCHAR(64) NOT NULL,
	CONSTRAINT pk_EmployeeAccount PRIMARY KEY (
		login
	 )
);

CREATE TABLE HeadOfDepartment (
	SSN VARCHAR(9) NOT NULL,
	CONSTRAINT pk_HeadOfDepartment PRIMARY KEY (
		SSN
	 ),
	employee_account VARCHAR(16) REFERENCES EmployeeAccount(login)
);

CREATE TABLE TaskOfToDoList (
	priority INTEGER NOT NULL,
	task_description VARCHAR(2048) NOT NULL,
	task_title VARCHAR(512) NOT NULL,
	task_status BOOLEAN NOT NULL,
	login VARCHAR(16) NOT NULL,
	CONSTRAINT pk_TaskOfToDoList PRIMARY KEY (
		task_title
	 ),
	CONSTRAINT fk_TaskOfToDoList_task_title FOREIGN KEY(login)
											REFERENCES EmployeeAccount (login)
);

CREATE TABLE Security (
	SSN VARCHAR(9) NOT NULL,
	CONSTRAINT pk_Security PRIMARY KEY (
		SSN
	 ),
	emplo

In [98]:
commands = full_text.split(";")
tables = {}

primary_key_expr = re.compile("PRIMARY KEY")
partial_key_expr = re.compile("FOREIGN KEY")
reference_expr = re.compile("REFERENCES")
        
for command in commands:
    try:
        table_name = command.split("CREATE TABLE")[1].split()[0]
    except:
        continue
        
    comma_sep = command.split("NOT NULL,")
    last = comma_sep[-1]
    comma_sep = comma_sep[:-1]
    comma_sep += last.split("),")

    first_sub_expr = comma_sep[0].split(table_name)[1]
    comma_sep[0] = re.sub(r"^\s+", "", first_sub_expr)[1:] # first brace removed

    table_col = []
    prim_keys = []
    partial_keys = []
    ref = {}          # column_name: TABLE_before
    
    for cs in comma_sep:
        cs = re.sub(r"^\s+", "", cs) # break by space
        print(cs)
        splited = cs.split()[:2]
        
        
        if primary_key_expr.search(cs):
            prim_keys_str = cs.split("(")[1].split(")")[0]
            prim_keys = prim_keys_str.split(",")
    
            prim_keys = [re.sub(r"\s+", "", pk) for pk in prim_keys]
        
        elif reference_expr.search(cs):
              
            arr = cs.split("REFERENCES")[1].split("(")
            ref_table = re.sub(r"\s+", "", arr[0])        # the name of table to search in
            ref_columns = arr[1].split(")")[0].split(",") # column(s) name(s) to search for
            ref_columns = [re.sub(r"\s+", "", rc) for rc in ref_columns]
            
            # add to dictionary
            for ref_column in ref_columns:
                ref[ref_column] = ref_table
                
            # add to array of exclusions
            if partial_key_expr.search(cs):
                partial_keys = ref_columns
        else:
            table_col += [splited]
        
    tables[table_name] = {"columns":table_col, 
                          "keys":prim_keys, 
                          "partial_keys": partial_keys, 
                          "ref": ref}

password VARCHAR(16) 
login VARCHAR(16) 
education VARCHAR(1024) 
date_of_start_of_career DATE 
surname VARCHAR(64) 
name VARCHAR(64) 
phone VARCHAR(11) 
email VARCHAR(64) 
CONSTRAINT pk_EmployeeAccount PRIMARY KEY (
		login
	 )
)
SSN VARCHAR(9) 
CONSTRAINT pk_HeadOfDepartment PRIMARY KEY (
		SSN
	 
employee_account VARCHAR(16) REFERENCES EmployeeAccount(login)
)
priority INTEGER 
task_description VARCHAR(2048) 
task_title VARCHAR(512) 
task_status BOOLEAN 
login VARCHAR(16) 
CONSTRAINT pk_TaskOfToDoList PRIMARY KEY (
		task_title
	 
CONSTRAINT fk_TaskOfToDoList_task_title FOREIGN KEY(login)
											REFERENCES EmployeeAccount (login)
)
SSN VARCHAR(9) 
CONSTRAINT pk_Security PRIMARY KEY (
		SSN
	 
employee_account VARCHAR(16) REFERENCES EmployeeAccount(login)
)
message_text VARCHAR(2048) 
title VARCHAR(512) 
employee_account VARCHAR(16) REFERENCES EmployeeAccount(login)
)
SSN VARCHAR(9) 
CONSTRAINT pk_HR PRIMARY KEY (
		SSN
	 
employee_account VARCHAR(16) REFERENCES EmployeeAccount(l

In [99]:
import json

column_names = np.array([])

for key in tables.keys():
    ARR_T = np.array(tables[key]["columns"])
    print(json.dumps(tables[key], indent=2))
    append = [arr_T[0] + "\t\t\t\t" + arr_T[1] for arr_T in ARR_T]
    column_names = np.append(column_names, append)
    
column_names = np.unique(column_names)
print("\n".join(column_names))

{
  "columns": [
    [
      "password",
      "VARCHAR(16)"
    ],
    [
      "login",
      "VARCHAR(16)"
    ],
    [
      "education",
      "VARCHAR(1024)"
    ],
    [
      "date_of_start_of_career",
      "DATE"
    ],
    [
      "surname",
      "VARCHAR(64)"
    ],
    [
      "name",
      "VARCHAR(64)"
    ],
    [
      "phone",
      "VARCHAR(11)"
    ],
    [
      "email",
      "VARCHAR(64)"
    ]
  ],
  "keys": [
    "login"
  ],
  "partial_keys": [],
  "ref": {}
}
{
  "columns": [
    [
      "SSN",
      "VARCHAR(9)"
    ]
  ],
  "keys": [
    "SSN"
  ],
  "partial_keys": [],
  "ref": {
    "login": "EmployeeAccount"
  }
}
{
  "columns": [
    [
      "priority",
      "INTEGER"
    ],
    [
      "task_description",
      "VARCHAR(2048)"
    ],
    [
      "task_title",
      "VARCHAR(512)"
    ],
    [
      "task_status",
      "BOOLEAN"
    ],
    [
      "login",
      "VARCHAR(16)"
    ]
  ],
  "keys": [
    "task_title"
  ],
  "partial_keys": [
    "login"

In [100]:
# parameters of random generation
MIN_YEAR = 1950

MIN_DOB = "1950-01-01"
MAX_DOB = "1995-01-01"

MIN_ROOM = 100
MAX_ROOM = 450

# random generators

def getRandomName():
    '''
    Returns random first name
    '''
    return names.get_first_name()

def getRandomSurname():
    '''
    Returns random surname
    '''
    return names.get_last_name()

def getRandomStringNoSpace(length):
    '''
    Returns randome string without spaces 
    (can be used for login-assword generation)
    '''
    return exrex.getone('[a-zA-Z][0-9a-zA-Z]{'+str(length-1)+'}')

def getRandomEmail():
    '''
    Random e-mail generation:
    - starts with small letter, 
    - then 7 small letter/numbers 
    - @
    - some small letter
    - .com or .ru
    
    Example of output: yfo5gtxf@u.com
    '''
    return exrex.getone('[a-z][a-z0-9]{7}@[a-z]\.(com|ru)$')

def getRandomDate(start=MIN_DOB, end=MAX_DOB):
    '''
    Returns date in format
    2018-08-21
    '''
    start, end = datetime.fromisoformat(start), datetime.fromisoformat(end)
    random_date = start + (end - start) * random.random()
    return random_date.date().__str__()

def getRandomDateTime(start=MIN_DOB+" 00:00:00", end=MAX_DOB+" 00:00:00"):
    '''
    Returns timestamp in format:
    2016-06-22 19:10:25
    '''
    start, end = datetime.fromisoformat(start), datetime.fromisoformat(end)
    random_date = start + (end - start) * random.random()
    return random_date.isoformat(" ", "seconds")
    
def getRandomString(length=64):
    '''
    Returns random string with multiple words 
    which can start from capital/small leters
    the length of the string is at max 'length'
    
    Can be used for long sentencies generation
    '''
    return exrex.getone('([a-zA-Z][a-z]* )*')[:length]

def getRandomPhone():
    '''
    Returns 11 numerical chars a row
    '''
    return exrex.getone('[0-9]{11}')

def getRandomBool():
    '''
    Returs "true" or "false"
    '''
    if random.randint(0, 1): return "true"
    return "false"

def getListOfRooms():
    '''
    Returns string 
    "ARRAY [{list}]", where list - sequence of numbers 
    from MIN_ROOM to MAX_ROOM of length from 1 to 6
    '''
    amount = np.random.randint(1, 6)
    result = [str(np.random.randint(MIN_ROOM, MAX_ROOM)) for x in range(amount)]
    result = ", ".join(result)
    result = "ARRAY [" + result + "]"
    return result

def getRandomMoney():
    '''
    Reurns float in format:
    xxxx.xx
    '''
    return np.random.randint(0, 10**6)/100

In [111]:
# dictionary of predefined fucions for some freuent column names


special_types = {"name":     lambda: getRandomName(), 
                 "surname":  lambda: getRandomSurname(), 
                 "email":    lambda: getRandomEmail(), 
                 "phone":    lambda: getRandomPhone(),
                 "SSN":      lambda: np.random.randint(10**9, 10**10),                  # 8 digits long
                 "medical_insurence_number": lambda: np.random.randint(10**16, 10**17), # 16 digits long
                 "year":     lambda: np.random.randint(MIN_YEAR, 2019), 
                 "season":   lambda: np.random.randint(1, 5), 
                 "month":    lambda: np.random.randint(1, 13),
                 "room":     lambda: np.random.randint(MIN_ROOM, MAX_ROOM),
                 "quantity": lambda: np.random.randint(0, 100),
                 "priority": lambda: np.random.randint(0, 10),
                 "login":    lambda: getRandomStringNoSpace(16), 
                 "password": lambda: getRandomStringNoSpace(16),
                 "list_of_rooms": lambda: getListOfRooms()
                }

# dictionary for includes

include_types = {"DATE":     lambda: getRandomDate(), 
                 "BOOLEAN":  lambda: getRandomBool(), 
                 "MONEY":    lambda: getRandomMoney()
                }

### What should I generate

```sql
INSERT INTO table(column1, column2)
VALUES (value1, value2);
```

In [112]:
MAIN_STR = ""
BD = {} # yes, it's possible to create even worse

def appendInsert(tableName, columns, values):
    '''
    Appends one more insert instruction to the global const
    MAIN_STR. Format of appending:
    (+) Updates local copy of bd
    
    "INSERT INTO {}({})\nVALUES ({});\n\n"
    '''
    global MAIN_STR
    str_columns, str_rows = ", ".join(columns), ", ".join(values)
    MAIN_STR += 'INSERT INTO {}({})\nVALUES ({});\n\n'.format(tableName, str_columns, str_rows)
    
    global BD
    for col, val in zip(columns, values):
        if tableName not in BD.keys():
            BD[tableName] = {}
        if col not in BD[tableName].keys():
            BD[tableName][col] = [val]
        BD[tableName][col].append(val)

In [113]:
tables

{'EmployeeAccount': {'columns': [['password', 'VARCHAR(16)'],
   ['login', 'VARCHAR(16)'],
   ['education', 'VARCHAR(1024)'],
   ['date_of_start_of_career', 'DATE'],
   ['surname', 'VARCHAR(64)'],
   ['name', 'VARCHAR(64)'],
   ['phone', 'VARCHAR(11)'],
   ['email', 'VARCHAR(64)']],
  'keys': ['login'],
  'partial_keys': [],
  'ref': {}},
 'HeadOfDepartment': {'columns': [['SSN', 'VARCHAR(9)']],
  'keys': ['SSN'],
  'partial_keys': [],
  'ref': {'login': 'EmployeeAccount'}},
 'TaskOfToDoList': {'columns': [['priority', 'INTEGER'],
   ['task_description', 'VARCHAR(2048)'],
   ['task_title', 'VARCHAR(512)'],
   ['task_status', 'BOOLEAN'],
   ['login', 'VARCHAR(16)']],
  'keys': ['task_title'],
  'partial_keys': ['login'],
  'ref': {'login': 'EmployeeAccount'}},
 'Security': {'columns': [['SSN', 'VARCHAR(9)']],
  'keys': ['SSN'],
  'partial_keys': [],
  'ref': {'login': 'EmployeeAccount'}},
 'SendMessage': {'columns': [['message_text', 'VARCHAR(2048)'],
   ['title', 'VARCHAR(512)']],
  'k

In [116]:
def getValueToInsert(column):
    
    varchar = re.compile("VARCHAR")
    timestamp = re.compile("timestamp")
    integer = re.compile("INTEGER")
    
    val = "__" # default value

    global special_types
    global include_types
    # if special column name
    if column[0] in special_types.keys():  
        val = special_types[column[0]]()

        
    # if special dt
    elif column[1] in include_types.keys():
        val = include_types[column[1]]()

    # if TIMESTAMP
    elif timestamp.search(column[0]):
        val = getRandomDateTime()

    # if VARCHAR(n)
    elif varchar.search(column[1]):
        amount = int(column[1].split("(")[1][:-1])
        val = getRandomString(amount)

    # if INTEGER dt    
    elif integer.search(column[1]):
        val = np.random.randint(0, 10**6)

    # check if everything is filled
    if val=='__':
        print ("Alert!", column, key)
    return str(val)


In [117]:
# introduction to KOSTILI:
# in any table ther is a column FULL_KEY (reserved column name), with stucked key values
def generateKey(table, column_names, keys):
    key_val = []
    for k in keys:
        val = getValueToInsert([k, table[column_names.index(k)][1]])
        key_val.append(val)
    return key_val

for table_name in tables.keys():
    # data for table     
    table = tables[table_name]["columns"]
    column_names = [column[0] for column in table] # names of usuall columns
    
    refs = tables[table_name]["ref"]
    keys = tables[table_name]["keys"]
    part_k = tables[table_name]["partial_keys"]
    
    for row in range(100):
        
        # fill connected columns
        ref_col, ref_val = [], []
        if len(refs) > 0:
            for k in refs.keys():
                col, val = k, np.random.choice(BD[refs[k]][k])
                ref_col.append(col)
                ref_val.append(val)
        
        # fill (partial) keys
        
        # arbitrary key
        if len(keys) > 0:
            key_val = generateKey(table, column_names, keys)
            print(table_name, key_val)
            should_be_unique = "".join(key_val)
        else:
            print("WTF", table_name)
        # partial
        par_val_str = ""
        par_val = []
        if len(part_k) > 0:
            for k in part_k:
                val = ref_val[ref_col.index(k)]
                par_val.append(val)
            par_val_str = "".join(par_val)
        
#         print(column_names)   
        # check uniquety
        i = 0
        if table_name in BD.keys():
            while should_be_unique + par_val_str in BD[table_name]["FULL_KEY"] and i<100:
                print(i, keys, table_name, should_be_unique)
                should_be_unique = "".join(generateKey(table, column_names, keys))
                i+=1
        
        # fill secondary columns
        functions = []
        k_counter = 0
        for column in table: # TODO: drop if key
            if column[0] not in keys:
                val = getValueToInsert(column)
                functions.append(str(val))
            else:
                functions.append(key_val[k_counter])
                k_counter += 1
            
        # create one more insertion instruction      
        appendInsert(table_name, 
                     column_names+ref_col+part_k+["FULL_KEY"], 
                     functions+ref_val+par_val+[should_be_unique+par_val_str])

EmployeeAccount ['NJEIYNn0qDOcABcR']
EmployeeAccount ['Mny5Jc482AjmAwm2']
EmployeeAccount ['ww2zGIkNPIuoPjRj']
EmployeeAccount ['tSwmXQKqDXdA0RPk']
EmployeeAccount ['CPNBh2Cat5W1HkOq']
EmployeeAccount ['x6w6O1jRnAgS7os5']
EmployeeAccount ['ALfno7LLupRAKftG']
EmployeeAccount ['L9kPiLn9I6iJw2Yn']
EmployeeAccount ['D5z3MMNKEt6VicTH']
EmployeeAccount ['hQnyWorEMr4mlW5B']
EmployeeAccount ['zuVkM2a726b4WJ03']
EmployeeAccount ['YU8b0wvRHI4B1Ytq']
EmployeeAccount ['AEqGbsmqsDuwYXnE']
EmployeeAccount ['mQUPdFXIZSOawWXH']
EmployeeAccount ['uzL7mgSYkZcns56O']
EmployeeAccount ['lF7AAxly4YkOZ8VV']
EmployeeAccount ['xULMH7wvu7LPWlgj']
EmployeeAccount ['EED8b8wmMxvBFvUA']
EmployeeAccount ['ENgVpzyHHn4LDgby']
EmployeeAccount ['di02GxqMrkTBJHy1']
EmployeeAccount ['c6PXddUZDiiwA0gv']
EmployeeAccount ['zv53yngpw8nx1UF1']
EmployeeAccount ['qhv66NWqu0YumAZS']
EmployeeAccount ['ASgyMMf6qZWjp5VK']
EmployeeAccount ['gauQt5BXKN8gg0cm']
EmployeeAccount ['CcBvFHJFCGoAtnqW']
EmployeeAccount ['IGPaTHKjrAHSHcYN']
E

Security ['1146706975']
Security ['6778653236']
Security ['6968145818']
Security ['5964101967']
Security ['7326882860']
Security ['2074990552']
Security ['7628110366']
Security ['3306873671']
Security ['1584166114']
Security ['6149938135']
Security ['7386491127']
Security ['9291825304']
Security ['9106764249']
Security ['2020780557']
Security ['8089128840']
Security ['4752986340']
Security ['4531750892']
Security ['4631449953']
Security ['2542491164']
Security ['7222780680']
Security ['4874579267']
Security ['8006436779']
Security ['1005019949']
Security ['6764557586']
Security ['1711849296']
Security ['2480514162']
Security ['6242716680']
Security ['1657583710']
Security ['1411149816']
Security ['8044818165']
Security ['4163420809']
Security ['5823588356']
Security ['9397245562']
Security ['9505109448']
Security ['1356114485']
Security ['1529039977']
Security ['3603509043']
Security ['7901059543']
Security ['7442596844']
Security ['1782893362']
Security ['6016562932']
Security ['54724

43 [] SendMessage 
44 [] SendMessage 
45 [] SendMessage 
46 [] SendMessage 
47 [] SendMessage 
48 [] SendMessage 
49 [] SendMessage 
50 [] SendMessage 
51 [] SendMessage 
52 [] SendMessage 
53 [] SendMessage 
54 [] SendMessage 
55 [] SendMessage 
56 [] SendMessage 
57 [] SendMessage 
58 [] SendMessage 
59 [] SendMessage 
60 [] SendMessage 
61 [] SendMessage 
62 [] SendMessage 
63 [] SendMessage 
64 [] SendMessage 
65 [] SendMessage 
66 [] SendMessage 
67 [] SendMessage 
68 [] SendMessage 
69 [] SendMessage 
70 [] SendMessage 
71 [] SendMessage 
72 [] SendMessage 
73 [] SendMessage 
74 [] SendMessage 
75 [] SendMessage 
76 [] SendMessage 
77 [] SendMessage 
78 [] SendMessage 
79 [] SendMessage 
80 [] SendMessage 
81 [] SendMessage 
82 [] SendMessage 
83 [] SendMessage 
84 [] SendMessage 
85 [] SendMessage 
86 [] SendMessage 
87 [] SendMessage 
88 [] SendMessage 
89 [] SendMessage 
90 [] SendMessage 
91 [] SendMessage 
92 [] SendMessage 
93 [] SendMessage 
94 [] SendMessage 
95 [] SendMe

55 [] SendMessage 
56 [] SendMessage 
57 [] SendMessage 
58 [] SendMessage 
59 [] SendMessage 
60 [] SendMessage 
61 [] SendMessage 
62 [] SendMessage 
63 [] SendMessage 
64 [] SendMessage 
65 [] SendMessage 
66 [] SendMessage 
67 [] SendMessage 
68 [] SendMessage 
69 [] SendMessage 
70 [] SendMessage 
71 [] SendMessage 
72 [] SendMessage 
73 [] SendMessage 
74 [] SendMessage 
75 [] SendMessage 
76 [] SendMessage 
77 [] SendMessage 
78 [] SendMessage 
79 [] SendMessage 
80 [] SendMessage 
81 [] SendMessage 
82 [] SendMessage 
83 [] SendMessage 
84 [] SendMessage 
85 [] SendMessage 
86 [] SendMessage 
87 [] SendMessage 
88 [] SendMessage 
89 [] SendMessage 
90 [] SendMessage 
91 [] SendMessage 
92 [] SendMessage 
93 [] SendMessage 
94 [] SendMessage 
95 [] SendMessage 
96 [] SendMessage 
97 [] SendMessage 
98 [] SendMessage 
99 [] SendMessage 
WTF SendMessage
0 [] SendMessage 
1 [] SendMessage 
2 [] SendMessage 
3 [] SendMessage 
4 [] SendMessage 
5 [] SendMessage 
6 [] SendMessage 
7 [

77 [] SendMessage 
78 [] SendMessage 
79 [] SendMessage 
80 [] SendMessage 
81 [] SendMessage 
82 [] SendMessage 
83 [] SendMessage 
84 [] SendMessage 
85 [] SendMessage 
86 [] SendMessage 
87 [] SendMessage 
88 [] SendMessage 
89 [] SendMessage 
90 [] SendMessage 
91 [] SendMessage 
92 [] SendMessage 
93 [] SendMessage 
94 [] SendMessage 
95 [] SendMessage 
96 [] SendMessage 
97 [] SendMessage 
98 [] SendMessage 
99 [] SendMessage 
WTF SendMessage
0 [] SendMessage 
1 [] SendMessage 
2 [] SendMessage 
3 [] SendMessage 
4 [] SendMessage 
5 [] SendMessage 
6 [] SendMessage 
7 [] SendMessage 
8 [] SendMessage 
9 [] SendMessage 
10 [] SendMessage 
11 [] SendMessage 
12 [] SendMessage 
13 [] SendMessage 
14 [] SendMessage 
15 [] SendMessage 
16 [] SendMessage 
17 [] SendMessage 
18 [] SendMessage 
19 [] SendMessage 
20 [] SendMessage 
21 [] SendMessage 
22 [] SendMessage 
23 [] SendMessage 
24 [] SendMessage 
25 [] SendMessage 
26 [] SendMessage 
27 [] SendMessage 
28 [] SendMessage 
29 [] 

...

82 [] ManagesWarehouseInventory 
83 [] ManagesWarehouseInventory 
84 [] ManagesWarehouseInventory 
85 [] ManagesWarehouseInventory 
86 [] ManagesWarehouseInventory 
87 [] ManagesWarehouseInventory 
88 [] ManagesWarehouseInventory 
89 [] ManagesWarehouseInventory 
90 [] ManagesWarehouseInventory 
91 [] ManagesWarehouseInventory 
92 [] ManagesWarehouseInventory 
93 [] ManagesWarehouseInventory 
94 [] ManagesWarehouseInventory 
95 [] ManagesWarehouseInventory 
96 [] ManagesWarehouseInventory 
97 [] ManagesWarehouseInventory 
98 [] ManagesWarehouseInventory 
99 [] ManagesWarehouseInventory 
WTF ManagesWarehouseInventory
0 [] ManagesWarehouseInventory 
1 [] ManagesWarehouseInventory 
2 [] ManagesWarehouseInventory 
3 [] ManagesWarehouseInventory 
4 [] ManagesWarehouseInventory 
5 [] ManagesWarehouseInventory 
6 [] ManagesWarehouseInventory 
7 [] ManagesWarehouseInventory 
8 [] ManagesWarehouseInventory 
9 [] ManagesWarehouseInventory 
10 [] ManagesWarehouseInventory 
11 [] ManagesWarehouseI

62 [] ManagesWarehouseInventory 
63 [] ManagesWarehouseInventory 
64 [] ManagesWarehouseInventory 
65 [] ManagesWarehouseInventory 
66 [] ManagesWarehouseInventory 
67 [] ManagesWarehouseInventory 
68 [] ManagesWarehouseInventory 
69 [] ManagesWarehouseInventory 
70 [] ManagesWarehouseInventory 
71 [] ManagesWarehouseInventory 
72 [] ManagesWarehouseInventory 
73 [] ManagesWarehouseInventory 
74 [] ManagesWarehouseInventory 
75 [] ManagesWarehouseInventory 
76 [] ManagesWarehouseInventory 
77 [] ManagesWarehouseInventory 
78 [] ManagesWarehouseInventory 
79 [] ManagesWarehouseInventory 
80 [] ManagesWarehouseInventory 
81 [] ManagesWarehouseInventory 
82 [] ManagesWarehouseInventory 
83 [] ManagesWarehouseInventory 
84 [] ManagesWarehouseInventory 
85 [] ManagesWarehouseInventory 
86 [] ManagesWarehouseInventory 
87 [] ManagesWarehouseInventory 
88 [] ManagesWarehouseInventory 
89 [] ManagesWarehouseInventory 
90 [] ManagesWarehouseInventory 
91 [] ManagesWarehouseInventory 
92 [] Mana

58 [] ManagesWarehouseInventory 
59 [] ManagesWarehouseInventory 
60 [] ManagesWarehouseInventory 
61 [] ManagesWarehouseInventory 
62 [] ManagesWarehouseInventory 
63 [] ManagesWarehouseInventory 
64 [] ManagesWarehouseInventory 
65 [] ManagesWarehouseInventory 
66 [] ManagesWarehouseInventory 
67 [] ManagesWarehouseInventory 
68 [] ManagesWarehouseInventory 
69 [] ManagesWarehouseInventory 
70 [] ManagesWarehouseInventory 
71 [] ManagesWarehouseInventory 
72 [] ManagesWarehouseInventory 
73 [] ManagesWarehouseInventory 
74 [] ManagesWarehouseInventory 
75 [] ManagesWarehouseInventory 
76 [] ManagesWarehouseInventory 
77 [] ManagesWarehouseInventory 
78 [] ManagesWarehouseInventory 
79 [] ManagesWarehouseInventory 
80 [] ManagesWarehouseInventory 
81 [] ManagesWarehouseInventory 
82 [] ManagesWarehouseInventory 
83 [] ManagesWarehouseInventory 
84 [] ManagesWarehouseInventory 
85 [] ManagesWarehouseInventory 
86 [] ManagesWarehouseInventory 
87 [] ManagesWarehouseInventory 
88 [] Mana

PatientAccount ['15592051298104205']
PatientAccount ['96050760937242758']
PatientAccount ['93082620364915758']
PatientAccount ['13616807480856994']
PatientAccount ['16964451564015300']
PatientAccount ['16042461982341792']
PatientAccount ['21544719188746879']
PatientAccount ['86831145970864335']
PatientAccount ['85724388593117078']
PatientAccount ['75828266540027898']
PatientAccount ['38925089777784675']
PatientAccount ['12095431843852064']
PatientAccount ['21207920938991332']
PatientAccount ['71726275559723518']
PatientAccount ['36047420119827040']
PatientAccount ['80165126093920536']
PatientAccount ['16035879370456759']
PatientAccount ['60501333605668575']
PatientAccount ['65810728075806026']
PatientAccount ['88683718686307060']
PatientAccount ['30323809789621359']
PatientAccount ['33804659645780949']
PatientAccount ['98687327236157959']
PatientAccount ['25347630143727820']
PatientAccount ['95523022014708636']
PatientAccount ['72750546311640842']
PatientAccount ['42008632513760617']
P

KeyError: 'Doctor'

In [11]:
# Printing the output to a file
out_file = open("fill.sql", "w")
out_file.write(MAIN_STR)
out_file.close()

## References

- [names generator](https://treyhunner.com/2013/02/random-name-generator/)
- [lib for regexp generator](https://github.com/asciimoo/exrex)
- [datetime](https://docs.python.org/3/library/datetime.html)
- [random date generation](https://cmsdk.com/python/generate-a-random-date-between-two-other-dates.html)

In [70]:
BD["EmployeeAccount"]

{'password': ['YnWs7LaY37b4KDq2',
  'YnWs7LaY37b4KDq2',
  'ZxrbeTaeOFMkBOA5',
  'mmUvqxXHjzrgMC6m',
  'iriszHbyh8naDbBG',
  'reWD8smYD6wwr5r4',
  'JHo5SnOnjmJ4KyWs',
  'H4KHKzUq8PNZJIsl',
  'ym6Xu18d51xtz91X',
  'mDIEYRbZdh6f389i',
  'uqppHojZeHmPrO9J',
  'NVj7ZqRyoJwgW9sA',
  'F8AlLtAZKhXVjj9W',
  'eJzI3p79zCGE9zfV',
  'am9QAjXvDnpcEtde',
  'fbqrUZMCdJqlzHhA',
  'eUYslHozKsBRFUxY',
  't7upkjk0RtymWs7W',
  'APBQNBtSdOyxRnBL',
  'SG4qD9sUjJ5myWie',
  'Uvug039rfAnNoEpB',
  'uu6bvOrZDfGM7Uux',
  'HdL1yfJKbk3bomEi',
  'm5oouuk1qnxGlNpz',
  'o2qdgsPynGK5AAIZ',
  'lZfiouWDZlKDznVp',
  'IC20j0irhOizXYMJ',
  'LkuP59mvRNkfMnLx',
  'kOxpFmStHTkfsbVt',
  'dQDnTBPMePZqY504',
  'FOG0THF0ig8CSc02',
  'IBUT5elz2slB0T98',
  'wAqpn1y6Vd7JqE70',
  'aDtssCIu6oPmILSo',
  'h9Zl801H1xqHUdd4',
  'ArEuzqnctxEaqqVr',
  'YvSrgEoFKHd8Wzfs',
  'eFHud4RyUjz6reEi',
  'aXIYLCxwnnWqnEeI',
  'wgfnmDSHnkaqUjdG',
  'hcUS66mgeK7B4CkC',
  'zj5KixADsui7qnAP',
  'QEgwavBbq0uk9aCt',
  'D4hgLRSyX5x59GJJ',
  'YrMSzT3yJAEBQkmc'